In [1]:
from pyspark import SparkContext

In [2]:
sc = SparkContext()

In [3]:
rdd_numeros = sc.parallelize([1,2,3,4])

In [4]:
duplicar = numeros.map(lambda x: x*2)

In [5]:
filtro = duplicar.filter(lambda x: x>5)

In [8]:
resultado = filtro.collect()
print("Resultado do filtro:", resultado)

Resultado do filtro: [6, 8]


In [ ]:
sc.stop()

In [19]:
rdd_ventas = sc.parallelize([("ProductoA",100),
                         ("ProductoB",23),
                         ("ProductoB",45),
                         ("ProductoA",10),
                         ("ProductoB",31),
                         ("ProductoB",16)])

In [20]:
ventas_por_producto = rdd_ventas.reduceByKey(lambda x, y: x + y)

In [23]:
orden_lista = ventas_por_producto.sortBy(lambda x: x[1], ascending=False)

In [24]:
resultado = orden_lista.collect()
print("Ventas totales por producto (de menor a mayor):")
for producto, total in resultado:
    print(f"{producto}: {total}")

Ventas totales por producto (de menor a mayor):
ProductoB: 115
ProductoA: 110


In [48]:
registros_rdd = sc.parallelize ([
    ("2024-11-10", "UPN-CHORRILLOS", "user1"),
    ("2024-11-10", "UPN-CHORRILLOS", "user2"),
    ("2024-11-10", "UPN-CHORRILLOS", "user3"),
    ("2024-11-11", "UPN-BREÑA", "user4"),
    ("2024-11-11", "UPN-CHORRILLOS", "user5"),
    ("2024-11-11", "UPN-BREÑA", "user1"),
    ("2024-11-12", "UPN-CHORRILLOS", "user6"),
    ("2024-11-12", "UPN-CHORRILLOS", "user2"),
    ("2024-11-12", "UPN-BREÑA", "user2"),
    ("2024-11-12", "UPN-CHORRILLOS", "user4"),
    ("2024-11-12", "UPN-CHORRILLOS", "user1"),
])

In [49]:
contador_accesos = registros_rdd.map(lambda x: (x[0], 1)).reduceByKey(lambda x, y: x + y)

In [50]:
resultado_dia = contador_accesos.collect()
print("Accesos por día:")
for fecha, total in resultado_dia:
    print(f"{fecha}: {total}")

Accesos por día:
2024-11-10: 3
2024-11-11: 3
2024-11-12: 5


In [51]:
sedes = registros_rdd.map(lambda x: (x[1], 1)).reduceByKey(lambda x, y: x + y)

In [52]:
orden_sede = sedes.sortBy(lambda x: x[1], ascending=False)

In [55]:
resultado_sede = orden_sede.collect()
print("Ingresos por sede (de mayor a menor):")
for sede, total in resultado_sede:
    print(f"{sede}: {total}")

Ingresos por sede (de mayor a menor):
UPN-CHORRILLOS: 8
UPN-BREÑA: 3


USARE UN ARCHIVO EXCEL

In [56]:
import pandas as pd

df = pd.read_excel("empresa_x2.xlsx")

In [57]:
datos = [tuple(x) for x in df.to_records(index=False)]

In [58]:
rdd_ventas = sc.parallelize(datos)

In [70]:
df

,Fecha,Producto o Servicio,Categoría,Cantidad Vendida,Precio Unitario,Total Venta
0,2024-01-01,Laptop Pro,Hardware,35,1079.96,37798.60
1,2024-01-01,Tablet Lite,Hardware,48,295.64,14190.72
2,2024-01-01,Software CRM,Software,47,1055.52,49609.44
3,2024-01-01,Soporte Técnico Mensual,Suscripción,5,739.12,3695.60
4,2024-02-01,Laptop Pro,Hardware,21,191.81,4028.01
5,2024-02-01,Tablet Lite,Hardware,18,357.72,6438.96
6,2024-02-01,Software CRM,Software,37,213.23,7889.51
7,2024-02-01,Soporte Técnico Mensual,Suscripción,34,950.94,32331.96
8,2024-03-01,Laptop Pro,Hardware,36,908.08,32690.88
9,2024-03-01,Tablet Lite,Hardware,42,882.44,37062.48


In [59]:
# Sumar las ventas totales por categoría
ventas_por_categoria = rdd_ventas.map(lambda x: (x[2], x[5])).reduceByKey(lambda x, y: x + y)

resultado_categoria = ventas_por_categoria.collect()
print("Ventas totales por categoría:")
for categoria, total in resultado_categoria:
    print(f"{categoria}: {total}")

Ventas totales por categoría:
Suscripción: 46883.52
Hardware: 179390.45
Software: 128929.99000000002


In [64]:
# Sumar la cantidad vendida por producto
cantidad_por_producto = rdd_ventas.map(lambda x: (x[1], x[3])).reduceByKey(lambda x, y: x + y)

orden_empresa = cantidad_por_producto.sortBy(lambda x: x[1], ascending=False)

resultado_producto = orden_empresa.collect()
print("Producto más vendido por cantidad:")
for producto, cantidad in resultado_producto:
    print(f"{producto}: {cantidad}")


Producto más vendido por cantidad:
Tablet Lite: 158
Laptop Pro: 122
Software CRM: 116
Soporte Técnico Mensual: 49
Software ERP: 40
Soporte Técnico Anual: 3


In [71]:
# Extraer el mes y sumar las ventas totales por mes
ventas_por_mes = rdd_ventas.map(lambda x:
 (pd.to_datetime(x[0]).strftime("%Y-%m"), x[5])).reduceByKey(lambda x, y: x + y)

ventas_ordenadas_mes = ventas_por_mes.sortByKey()

resultado_mes = ventas_ordenadas_mes.collect()
print("Ventas totales por mes:")
for mes, total in resultado_mes:
    print(f"{mes}: {total}")

Ventas totales por mes:
2024-01: 105294.36000000002
2024-02: 50688.44
2024-03: 199221.15999999997
